In [1]:
!pip install google-api-python-client
!python -c "from oauth2client import crypt"
!pip install pyCrypto

     |████████████████████████████████| 446 kB 29.8 MB/s 
  Created wheel for pyCrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=69cda63c6c262146b078105e8727981cff532a5ea44043d4f991877ecf0754e8
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pyCrypto


#Connecting GEE with other Google Services

In [2]:
!pip install earthengine-api
!pip install geojson
!pip install geopandas

     |████████████████████████████████| 994 kB 16.8 MB/s 
     |████████████████████████████████| 6.3 MB 49.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 


#**Authentication**

To read/write from a Google Drive or Google Cloud Storage bucket to which you have access, it's necessary to authenticate (as yourself). You'll also need to authenticate as yourself with Earth Engine, so that you'll have access to your scripts, assets, etc.

Authenticate to Earth Engine
In order to access Earth Engine, signup at signup.earthengine.google.com.

In [3]:
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=iZgVRj94huK621IAcBVu6DFL5TB4MZZeV6VxTZrV7ko&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiuYYYKI60KmLpUYlJgWMz0zwH2Mj0AB_UL69r78-3Ke2_woil9s7Y

Successfully saved authorization token.


#**Authenticate Google Drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Authenticate to Google Cloud**

In [6]:
from google.colab import auth
auth.authenticate_user()


# **Import earth engine API and initialize it**

As you can see in the description of the datasets, they include several sets of information stored in several bands. For example, these bands are associated with the LST collection:

LST_Day_1km: Daytime Land Surface Temperature

Day_view_time: Local time of day observation

LST_Night_1km: Nighttime Land Surface Temperature
etc.

In [7]:
import ee
ee.Initialize()
import folium
import webbrowser

%matplotlib inline
import time 
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import geojson
import requests
import geopandas as gpd
import time
import numpy as np

# import rasterio as rio
# from rasterio.plot import show
# import rasterio.mask

# from rasterio import windows
# from shapely.geometry import box
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
%matplotlib inline
# import ee
# import geemap
# import geemap.eefolium as emap
# import folium

# **Fetch Netherlands shape file from drive**

In [8]:
netherlands_provinces = '/content/drive/MyDrive/Netherland Data/NL-new-singlepolygon.geojson'

In [9]:
provinces = gpd.read_file(netherlands_provinces)
provinces.head(10)

,id,id_0,iso,name_0,id_1,name_1,hasc_1,ccn_1,cca_1,type_1,engtype_1,nl_name_1,varname_1,geometry
0,st293bj4601.1,158,NLD,Netherlands,1,Drenthe,NL.DR,0,None,Provincie,Province,None,None,"POLYGON ((6.50131 53.19841, 6.50429 53.19749, ..."
1,st293bj4601.10,158,NLD,Netherlands,10,Overijssel,NL.OV,0,None,Provincie,Province,None,None,"POLYGON ((6.13809 52.84380, 6.15725 52.83023, ..."
2,st293bj4601.2,158,NLD,Netherlands,2,Flevoland,NL.FL,0,None,Provincie,Province,None,None,"POLYGON ((5.44521 52.55581, 5.43211 52.53638, ..."
3,st293bj4601.2,158,NLD,Netherlands,2,Flevoland,NL.FL,0,None,Provincie,Province,None,None,"POLYGON ((5.44521 52.55581, 5.45261 52.56209, ..."
4,st293bj4601.2,158,NLD,Netherlands,2,Flevoland,NL.FL,0,None,Provincie,Province,None,None,"POLYGON ((5.85673 52.55246, 5.85966 52.54976, ..."
5,st293bj4601.2,158,NLD,Netherlands,2,Flevoland,NL.FL,0,None,Provincie,Province,None,None,"POLYGON ((5.46890 52.56568, 5.47186 52.56837, ..."
6,st293bj4601.2,158,NLD,Netherlands,2,Flevoland,NL.FL,0,None,Provincie,Province,None,None,"POLYGON ((5.75513 52.83910, 5.77305 52.82814, ..."
7,st293bj4601.3,158,NLD,Netherlands,3,Friesland,NL.FR,0,None,Provincie,Province,None,Frise|Frisia|Frísia,"POLYGON ((5.24917 53.25458, 5.24917 53.25431, ..."
8,st293bj4601.3,158,NLD,Netherlands,3,Friesland,NL.FR,0,None,Provincie,Province,None,Frise|Frisia|Frísia,"POLYGON ((5.13750 53.29930, 5.13750 53.29903, ..."
9,st293bj4601.3,158,NLD,Netherlands,3,Friesland,NL.FR,0,None,Provincie,Province,None,Frise|Frisia|Frísia,"POLYGON ((5.07028 53.30708, 5.07028 53.30680, ..."


In [10]:
provinces['name_1'].unique()

array(['Drenthe', 'Overijssel', 'Flevoland', 'Friesland', 'Gelderland',
       'Groningen', 'IJsselmeer', 'Limburg', 'Noord-Brabant',
       'Noord-Holland', 'Utrecht', 'Zeeland', 'Zeeuwse meren',
       'Zuid-Holland'], dtype=object)

In [11]:
province_centroids = {}
for province in provinces.name_1.unique():
  if province == 'Caribbean Netherlands':
    continue
  centroid = provinces[provinces.name_1 == province].centroid.iloc[0].coords[0]
  province_centroids[province] = centroid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """


In [12]:
province_centroids

{'Drenthe': (6.623756376076824, 52.863148945531755),
 'Flevoland': (5.434529938005638, 52.52925461942454),
 'Friesland': (5.253107926490933, 53.25172118970759),
 'Gelderland': (5.945177854611231, 52.061725686144385),
 'Groningen': (6.744453751533158, 53.21709289600278),
 'IJsselmeer': (5.333586194277391, 52.68827345307856),
 'Limburg': (5.936317658053973, 51.20757911171452),
 'Noord-Brabant': (5.198520546903542, 51.56034032493212),
 'Noord-Holland': (4.998244336561861, 52.35598135256618),
 'Overijssel': (6.447599731120618, 52.4451359713196),
 'Utrecht': (5.177352142850485, 52.09596543653061),
 'Zeeland': (3.786650684511968, 51.312778733455666),
 'Zeeuwse meren': (4.105014281767546, 51.62144851898593),
 'Zuid-Holland': (4.309252026225199, 51.73949711049258)}

In [13]:
centroids= []
for key,value in province_centroids.items():
  centroids.append(value)

In [14]:
area_netherland = ee.Geometry.Polygon(centroids)
area_netherland.getInfo()

{'coordinates': [[[6.623756376076824, 52.863148945531755],
   [6.447599731120618, 52.4451359713196],
   [5.434529938005638, 52.52925461942454],
   [5.253107926490933, 53.25172118970759],
   [5.945177854611231, 52.061725686144385],
   [6.744453751533158, 53.21709289600278],
   [5.333586194277391, 52.68827345307856],
   [5.936317658053973, 51.20757911171452],
   [5.198520546903542, 51.56034032493212],
   [4.998244336561861, 52.35598135256618],
   [5.177352142850485, 52.09596543653061],
   [3.786650684511968, 51.312778733455666],
   [4.105014281767546, 51.62144851898593],
   [4.309252026225199, 51.73949711049258],
   [6.623756376076824, 52.863148945531755]]],
 'type': 'Polygon'}

In [15]:
# # Define the geometry of the area for which you would like images.
# geom = ee.Geometry.Polygon([[33.8777, -13.4055],
#                             [33.8777, -13.3157],
#                             [33.9701, -13.3157],
#                             [33.9701, -13.4055]])
# area_geometry= ee.Geometry.Point(5.2913,52.1326)

#**Generate and export Image from MODIS Satellite**

In [20]:
# Initial date of interest (inclusive).
i_date = '2020-08-01'

# Final date of interest (exclusive).
from datetime import date

f_date = str(date.today())

In [21]:
netharland_collection= ee.ImageCollection('MODIS/006/MOD11A1')
netharland_collection_lst = netharland_collection.select(['LST_Day_1km','LST_Night_1km']).filterDate(i_date, f_date)

print('Number of Images:',int(netharland_collection_lst.size().getInfo()))

Number of Images: 396


In [22]:
cnt= netharland_collection_lst.size().getInfo()
cnt

396

In [23]:
netharland_collection_lst.getInfo()

{'bands': [],
 'features': [{'bands': [{'crs': 'SR-ORG:6974',
     'crs_transform': [926.625433055833,
      0,
      -20015109.353988,
      0,
      -926.6254330549998,
      10007554.676994],
     'data_type': {'max': 65535,
      'min': 0,
      'precision': 'int',
      'type': 'PixelType'},
     'dimensions': [43200, 21600],
     'id': 'LST_Day_1km'},
    {'crs': 'SR-ORG:6974',
     'crs_transform': [926.625433055833,
      0,
      -20015109.353988,
      0,
      -926.6254330549998,
      10007554.676994],
     'data_type': {'max': 65535,
      'min': 0,
      'precision': 'int',
      'type': 'PixelType'},
     'dimensions': [43200, 21600],
     'id': 'LST_Night_1km'}],
   'id': 'MODIS/006/MOD11A1/2020_08_01',
   'properties': {'system:asset_size': 782293872,
    'system:footprint': {'coordinates': [[-180, -90],
      [180, -90],
      [180, 90],
      [-180, 90],
      [-180, -90]],
     'type': 'LinearRing'},
    'system:index': '2020_08_01',
    'system:time_end': 159632640

In [24]:
# # Sort images in the collection by index (which is equivalent to sorting by date), 
# # with the oldest images at the front of the collection.
# # Convert collection into a single image mosaic where only images at the top of the collection are visible.
# image= netharland_collection_lst.sort('system:index',opt_ascending=False).mosaic()
#palette=['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', 
                                                  #  '99B718','74A901','66A000', '529400', '3E8601', 
                                                  #  '207401', '056201', '004C00','023B01','012E01', '011D01', '011301']

                                                  # 'crs':'EPSG:1133'
#palette parameters only work when single band parameter is there.
# ,palette=['aec3d4',  # water
#     '152106',
#     '225129',
#     '369b47',
#     '30eb5b',
#     '387242',  # forest
#     '6a2325',
#     'c3aa69',
#     'b76031',
#     'd9903d',
#     '91af40',  # shrub, grass
#     '111149',  # wetlands
#     'cdb33b',  # croplands
#     'cc0013',  # urban
#     '33280d',  # crop mosaic
#     'd7cdcc',  # snow and ice
#     'f7e084',  # barren
#     '6f6f6f'])  # tundra

In [25]:
#Visualization Parameter
image= netharland_collection_lst.sort('system:index',opt_ascending=False).mosaic()
image= image.visualize(bands=['LST_Day_1km'],min=500,max=15000,palette=['00FFFF', '0000FF'])



In [26]:
image_list = netharland_collection.toList(cnt)

# **Exporting Image to Drive**

In [ ]:
for i in range(0, int(netharland_collection_lst.size().getInfo())):
  image = ee.Image(image_list.get(i))
  print('i: ', i)
  name = image.get('system:index').getInfo()
  image= image.visualize(bands=['LST_Day_1km','LST_Night_1km'],min=500,max=15000)
  print('Name: ', name)
  task = ee.batch.Export.image.toDrive(**{
    'image': image,
    'description': 'image_' + str(i),
    'folder':'MODIS_satellite_images_2020_to_2021',
    'scale': 100,
    'region': area_netherland.coordinates().getInfo(),
    'crs':'EPSG:23095',
    'fileFormat': 'GeoTIFF'

  })
  task.start()
  print(task.status())
  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(3)
  print(task.status())

Streaming output truncated to the last 5000 lines.
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (id: GHWNWMXSBXOY2ESMWABQ7CLC).
Polling for task (

# **Exporting Image via URL**

In [198]:
def get_url(name, image, scale, region):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    If additional parameters are needed, see also:
    https://github.com/google/earthengine-api/blob/master/python/ee/image.py

    Parameters:
        name (str): name of the created folder
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest

    Returns:
        path (str)
     """
    path= image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'crs': 'EPSG:23095',
        'region': region
    })
    webbrowser.open_new_tab(path)
    return path

In [ ]:
for i in range(0, int(netharland_collection_lst.size().getInfo())):
  image = ee.Image(image_list.get(i))
  print('i: ', i)
  name = image.get('system:index').getInfo()
  image= image.visualize(bands=['LST_Day_1km','LST_Night_1km'],min=500,max=15000)
  print('Name: ', name)
  url= get_url(name,image,100,region_netherland)
  print(url)

i:  0
Name:  2000_02_24
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fbf53ded5b8a24dd90290f26de2e29d0-2d000e46599c7ec2fbc04faeee1ee2db:getPixels
i:  1
Name:  2000_02_25
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6a1a5faf949b296055937f9f4e1d8e5c-0c5b6dcc35ac01c118ce39512117543a:getPixels
i:  2
Name:  2000_02_26
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/bc0d96207e37f4c0620f5ea4947eb898-ba9373144fd0cbf8112980ebae766235:getPixels
i:  3
Name:  2000_02_27
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2a9f5c878c8c7d11898ac67cfb9c92b8-97721dda4bdf01ba2cfc77a37389872f:getPixels
i:  4
Name:  2000_02_28
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2810c1988108fde8925bd0cab6e1f44c-41b6a428af9f3b3cba38db2e0b3bcd53:getPixels
i:  5
Name:  2000_02_29
https://earthengine.googleapis.com/v1alpha/projects/earthengine-le

In [ ]:
# task_config= {'region': area_netherland.coordinates().getInfo(),'folder': 'MODISImages','scale': 100,'crs':'EPSG:23095'}
# task= ee.batch.Export.image.toDrive(image,**task_config)
# task.start()

In [169]:
# task.status()